In [35]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [64]:
def predictions(clf, X_test, scaler=None):
    if not scaler == None:
        X_test   = scaler.transform(X_test)
    y_hat    = clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    return y_hat, y_hat_pr

In [311]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    
    clf = LogisticRegressionCV(max_iter=1000, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test, scaler)
    
    return clf, scaler, y_hat, y_hat_pr

In [312]:
def RF(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat, y_hat_pr = predictions(clf, X_test)
    
    return clf, y_hat, y_hat_pr

In [313]:
def prepare_datasets(nor, adv, factor=0.8):
    if len(nor.shape) > 2: 
        nor = nor.reshape((nor.shape[0], -1))
        adv = adv.reshape((adv.shape[0], -1))

    y_nor = np.zeros(nor.shape[0]).astype('int')
    y_adv = np.ones(adv.shape[0]).astype('int')

    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

    X_train = np.concatenate((x_train_n, x_train_a))
    y_train = np.concatenate((y_train_n, y_train_a))

    X_test = np.concatenate((x_test_n, x_test_a))
    y_test = np.concatenate((y_test_n, y_test_a))

    return X_train, y_train, X_test, y_test

In [ ]:
factor = 0.6
nr_mean = 2
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
# detectors = ['lid', 'multilid']
detectors = ['multilid']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    results[it] = {}

    for dataset in datasets:
        if dataset == 'imagenet':
            k=30
        else:
            k=20

        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue

            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}

            for detector in detectors:
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))[:nr_samples]
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))[:nr_samples]
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))[:nr_samples]
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))[:nr_samples]
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))[:nr_samples]
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))[:nr_samples]
                
                assert(nor_fgsm.shape[0] == nr_samples)
                assert(nor_bim.shape[0] == nr_samples)
                assert(nor_pgd.shape[0] == nr_samples)
                assert(nor_aa.shape[0] == nr_samples)
                assert(nor_df.shape[0] == nr_samples)
                assert(nor_cw.shape[0] == nr_samples)

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for source in range(len(nor)):
                    X_train_src, y_train_src, X_test_src, y_test_src = prepare_datasets(nor[source], adv[source], factor)
                    clf_lr, scaler, y_hat_lr, y_hat_pr_lr = LR(X_train_src, y_train_src, X_test_src, y_test_src)
                    clf_rf, y_hat_rf, y_hat_pr_rf = RF(X_train_src, y_train_src, X_test_src, y_test_src)

                    for target in range(len(nor)):
                        if source == target:
                            continue
                        
                        X_train_tar, y_train_tar, X_test_tar, y_test_tar = prepare_datasets(nor[target], adv[target], factor)
                        
                        y_test_lr, y_hat_pr_lr = predictions(clf_lr, X_test_tar, scaler)
                        auc_lr = round(100*roc_auc_score(y_test_lr, y_hat_pr_lr), 2)
                        f1_lr =  round(100*f1_score(y_test_lr, y_hat_lr), 2)

                        y_test_rf, y_hat_pr_rf = predictions(clf_rf, X_test_tar)
                        auc_rf = round(100*roc_auc_score(y_test_rf, y_hat_pr_rf), 2)
                        f1_rf =  round(100*f1_score(y_test_rf, y_hat_rf), 2)
                        
                        if not detector in results[it][dataset][model]:
                            results[it][dataset][model][detector] = {}

                        if not attacks[source] in results[it][dataset][model][detector]: 
                            results[it][dataset][model][detector][attacks[source]] = {}

                        if not attacks[target] in results[it][dataset][model][detector][attacks[source]]: 
                            results[it][dataset][model][detector][attacks[source]][attacks[target]] = {}
            
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['auc_lr'] = auc_lr
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['auc_rf'] = auc_rf
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['f1_lr']  = f1_lr
                        results[it][dataset][model][detector][attacks[source]][attacks[target]]['f1_rf']  = f1_rf

# LID

# multiLID

In [299]:
# source_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
source_attacks = ['fgsm', 'bim']
target_attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
datasets =  ['cifar10', 'cifar100', 'imagenet']

lid_list_mean = []

for it_mean in range(nr_mean):
    lid_list = []

    for dataset in datasets:
        for model in models:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            for source_attack in source_attacks:
                tmp = np.zeros((len(target_attacks)-1, 2))
                iter_tar = 0
                
                for target_attack in target_attacks:
                    if target_attack == source_attack:
                        continue
                        
                    for it, val in enumerate(results[it_mean][dataset][model]['multilid'][source_attack][target_attack].items()):
                        tmp[iter_tar, it] = val[1]
                    iter_tar += 1
                
                lid_list.append(tmp.copy())
    lid_list_mean.append(np.stack(lid_list))

lid_mean = np.mean(lid_list_mean,axis=0)
lid_var = np.var(lid_list_mean,axis=0)

In [300]:
lid_mean.shape

(10, 5, 2)

In [305]:
attacks_saved

['bim', 'pgd', 'aa', 'df', 'cw']

In [310]:
latex_tables = []

for it_src, source in enumerate(source_attacks):
    latex_table = ""
    attacks_saved = target_attacks.copy()
    attacks_saved.remove(source)
    print(source, attacks_saved)
    for j in range(lid_mean.shape[1]):
        latex_table += "\\textbf{"+f"{source.upper()}"+"} & \\textbf{"+f"{attacks_saved[j].upper()}"+"}"
        for i in range(lid_mean.shape[1]):
            mean = lid_mean[it_src*len(attacks_saved)+i, j, 0]
            variance = lid_var[i, j, 0]
            latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
            mean = lid_mean[it_src*len(attacks_saved)+i, j, 1]
            variance = lid_var[i, j, 1]
            latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        latex_table = latex_table[:-2] + "\\\\\n"
    latex_table += "\n\n"
    print(latex_table)
    latex_tables.append(latex_table)

fgsm ['bim', 'pgd', 'aa', 'df', 'cw']
\textbf{FGSM} & \textbf{BIM}$80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ \\
\textbf{FGSM} & \textbf{PGD}$80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ \\
\textbf{FGSM} & \textbf{AA}$80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ \\
\textbf{FGSM} & \textbf{DF}$80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.03 \pm 0.00$ \\
\textbf{FGSM} & \textbf{CW}$80.32 \pm 0.00$ & $70.03 \pm 0.00$ & $80.32 \pm 0.00$ & $70.